In [ ]:
# Check S3 connection directory
!ls /datasets/shopee-week2

shopee-product-detection-dataset.zip


In [ ]:
idx = 22
premodel_name = 'effnetb7'
optimizer = 'adam'
is_class_weight = True
dropout = 0.2
dense = 256
epochs = 10
image_size = 299
batch_size = 32

In [ ]:
import numpy as np
import pandas as pd
import zipfile
import os
import matplotlib.pyplot as plt
import glob
from time import time
%matplotlib inline

import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from efficientnet.tfkeras import EfficientNetB4, EfficientNetB7
from sklearn.utils.class_weight import compute_class_weight


physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print("Tensorflow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print("Is using GPU?", gpus)

tf.config.experimental.set_memory_growth(gpus[0], True)

Tensorflow version: 2.2.0
Is using GPU? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Import and organize files

In [ ]:
# Check if the zip file has been extracted. Otherwise, extract from S3 connection
copyflag=False
if not (os.path.isdir("train/train") and os.path.isdir("test/test")):
    with zipfile.ZipFile("/datasets/shopee-week2/shopee-product-detection-dataset.zip", "r") as zfile:
        zfile.extractall(".")
    copyflag=True
# #
# get path return generator of files
def files(path):
    '''
    Return generator of files in a given path

    argument: path
    return: file
    '''
    
    for f in os.listdir(path):
        if os.path.isfile(os.path.join(path, f)):
            yield f
# if there are still subfolders, move them to the train folder
if copyflag: # do moving only if the files are freshly extracted
    for dirpath, dirnames, filenames in os.walk("train/train", topdown=False):
        # walh through the training folder
        if dirnames: # if folder exists
            for d in dirnames:
                for f in files(d): # move all files inside folder to training folder
                    os.replace(os.path.join(d, f), os.path.join("train/train", f))

# remove empty folders after moving
def drop_empty_folders(path):
    '''
    remove empty subfolders in a given path

    argument: path
    return: none
    '''
    for dirpath, dirnames, filenames in os.walk(path, topdown=False):
        if not dirnames and not filenames:
            os.rmdir(dirpath)
drop_empty_folders("train/train")


# # check number of extracted files
train_files = os.listdir("train/train")
test_files = os.listdir("test/test")

df_train = pd.read_csv("train.csv", dtype=str)
df_test = pd.read_csv("test.csv", dtype=str)

# check if number of image files is equal to number of index in csv
print(f"Number of training images: {len(train_files)}")
print(f"Number of training indices: {df_train.shape[0]}")
print(f"Number of testing images: {len(test_files)}")
print(f"Number of testing indices: {df_test.shape[0]}")



Number of training images: 105398
Number of training indices: 105392
Number of testing images: 12192
Number of testing indices: 12186


In [ ]:
def remove_tmp_files(path, img_files):
    '''
    remove files ending with .tmp in a given path

    argument: path, list of filenames
    return: list of filenames
    '''
    print(f"Inspecting path: {path}")
    for f in glob.glob(os.path.join(path, '*.tmp')):
        f = f.rsplit('/')[-1]
        try:
            img_files.remove(f)
            print(f"Removing file: {f}")
        except:
            print(f"Trying to remove non-existing file: {f}")
    return img_files
train_files = remove_tmp_files('train/train', train_files)
test_files = remove_tmp_files('test/test', test_files)

# check again if number of image files is equal to number of index in csv
print(f"Number of training images: {len(train_files)}")
print(f"Number of training indices: {df_train.shape[0]}")
print(f"Number of testing images: {len(test_files)}")
print(f"Number of testing indices: {df_test.shape[0]}")

Inspecting path: train/train
Removing file: b4c1a220fd3a43961017717d953fb4e5.jpgk4d4lnxk.tmp
Removing file: 6b6e7a2db5acf395ee36077808f32e3d.jpgt13z3ib7.tmp
Removing file: 8235bf4e577b890f5a5ecfda00951f84.jpgt3reyw1v.tmp
Removing file: a6f9b9e65c3576a64452961a21678395.jpg95ta80mt.tmp
Removing file: 4af102e634973b11e5ab90cdee121a6d.jpgt0581za2.tmp
Removing file: 4e658733865eebe2461c322a54379e7f.jpg8afrrohw.tmp
Inspecting path: test/test
Number of training images: 105392
Number of training indices: 105392
Number of testing images: 12192
Number of testing indices: 12186


# Get random batch

In [ ]:
image_dir = 'train/train/'
image_size = image_size # [96, 128, 160, 192, 224]

def get_random_batch(df, batch_size=4):
    
    indices = np.random.choice(range(df.shape[0]), batch_size, replace=False)
    x = np.zeros((batch_size, image_size, image_size, 3))
    y = np.zeros((batch_size, 1))
    images = []

    for i, index in enumerate(indices):
        image = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, df.loc[index]['filename']), 
                                                        target_size=(image_size,image_size))
        images.append(image)
        arr = tf.keras.preprocessing.image.img_to_array(image)
        arr = tf.keras.applications.mobilenet_v2.preprocess_input(arr)
        arr = np.expand_dims(arr, axis=0)
        x[i] = arr
        y[i] = int(df.loc[index]['category'])
    return x, y, images

def display_examples(x, y, p, images, df):

    print('Displaying first 8 examples..')

    if len(images) < 8:
        print('Need at least 8 examples')
        return None

    plt.figure(figsize=(12, 8))
    for i in range(8):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        plt.xticks([])
        plt.yticks([])
        col = 'green' if y[i][0]==p[i][0] else 'red'
        plt.xlabel(p[i][0], color=col)
    return plt

In [ ]:
x, y, images = get_random_batch(df_train, batch_size=8)
display_examples(x, y, y, images, df_train).show()

# Preprocess data


## Get random eraser

In [ ]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [ ]:
disp_eraser = get_random_eraser(v_l = 0., v_h = 255., pixel_level=True, p=1)
x, y, images = get_random_batch(df_train, batch_size=8)
display_examples(x, y, y, [disp_eraser(np.array(im)) for im in images], df_train).show()


# Splitting train and val set

In [ ]:
def splitting_data(df, train_ratio=0.9):
    indices = np.random.choice(df.shape[0], int(train_ratio*df.shape[0]), replace=False)
    remaining = set(df.index) - set(indices)
    df_model = df.loc[indices].reset_index()
    df_val = df.loc[remaining].reset_index()
    return df_model, df_val
df_model, df_val = splitting_data(df_train, train_ratio=0.90)
print("Original shape: ", df_train.shape[0])
print("df_model shape: ", df_model.shape[0])
print("df_val shape: ", df_val.shape[0])

In [ ]:
# compare density of splitting categories
fig = plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(df_model['category'], density=True, bins=42)
plt.title("y_train")
plt.subplot(1,2,2)
plt.hist(df_val['category'], density=True, bins=42)
plt.title("y_val")
plt.show()

In [ ]:
cat_cols = df_model['category'].unique()
if is_class_weight == True:
    class_weight = compute_class_weight('balanced',
                                        np.unique(df_model['category']),
                                        df_model['category'])
    class_weight = dict(enumerate(class_weight))
else:
    class_weight = {c: 1 for c in range(cat_cols.shape[0])}
class_weight

# Create model

In [ ]:
if premodel_name == 'mnet':
    premodel = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top=False, input_shape=(image_size, image_size, 3),
        pooling='avg', weights='imagenet')

elif premodel_name == 'effnetb4':
    premodel = EfficientNetB4(
        include_top=False, input_shape=(image_size, image_size, 3),
        pooling='avg', weights='imagenet'
    )

elif premodel_name == 'effnetb7':
    premodel = EfficientNetB7(
        include_top=False, input_shape=(image_size, image_size, 3),
        pooling='avg', weights='imagenet'
    )
premodel.summary()

In [ ]:
def create_model():
    model = tf.keras.models.Sequential([
        premodel,
        tf.keras.layers.Dense(dense, activation='relu'),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(dense, activation='relu'),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(42, activation='softmax')
    ])

    model.layers[0].trainable = False
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
    return model

model = create_model()
model.summary()

# Model training

In [ ]:

batch_size = batch_size
steps_per_epoch = int(df_model.shape[0]/batch_size)
validation_steps = int(df_val.shape[0]/batch_size)

print('Steps per epoch:', steps_per_epoch)
print('Validation steps:', validation_steps)


def preprocess_input(img):
    
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    eraser = get_random_eraser(v_l=-1., v_h=1., pixel_level=True, p=0.5)
    img = eraser(img)
    return img

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_dataframe(
    df_model,
    directory=image_dir,
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(image_size, image_size),
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

validation_generator = validation_datagen.flow_from_dataframe(
    df_val,
    directory=image_dir,
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(image_size, image_size),
    batch_size=batch_size
)

In [ ]:

start_time = time()
h = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    verbose=1,
    epochs=epochs,
    class_weight=class_weight, 
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=2),
        tf.keras.callbacks.ModelCheckpoint('models/model_{val_acc:.3f}.h5', save_best_only=True,
                                          save_weights_only=False, monitor='val_acc')
    ]

    )


print("Wall time: ", time()-start_time)


In [0]:
losses = h.history['loss']
accs = h.history['acc']
val_losses = h.history['val_loss']
val_accs = h.history['val_acc']
epochs = len(losses)

plt.figure(figsize=(12, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

print("Loss: {:.4f}".format(losses[-1]))
print("Acc: {:.4f}".format(accs[-1]))
print("Val Loss: {:.4f}".format(val_losses[-1]))
print("Val Accs: {:.4f}".format(val_accs[-1]))

# Visualize validations

In [0]:
x, y, images = get_random_batch(df_val, batch_size=8)
preds = model.predict(x).argmax(axis=1).reshape(-1,1)
display_examples(x, y, preds, images, df_val).show()

# Make prediction   

In [0]:
# model.load_weights('models/model_0.686.h5')

In [0]:
def create_prediction(df, batch_size=8, image_dir='test/test'):
    
    y = np.zeros((df.shape[0], 1))
    steps = int(df.shape[0]/batch_size)
    for i in range(0, steps+1):
        df_sub = df.loc[i*batch_size: (i+1)*batch_size].reset_index()
        if i == steps:
            batch_size = df.shape[0] % batch_size
        x = np.zeros((batch_size, image_size, image_size, 3))
        for j in range(0, batch_size):
            image = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, df_sub.loc[j,'filename']), 
                                                            target_size=(image_size,image_size))
            arr = tf.keras.preprocessing.image.img_to_array(image)
            arr = tf.keras.applications.mobilenet_v2.preprocess_input(arr)
            arr = np.expand_dims(arr, axis=0)
            x[j] = arr
        if i == steps:
            y[-batch_size:] = model.predict(x).argmax(axis=1).reshape(-1,1)
        else:
            y[i*batch_size: (i+1)*batch_size] = model.predict(x).argmax(axis=1).reshape(-1,1)
    df['category'] = y
    df['category'] = df['category'].apply(lambda v: str(int(v)).zfill(2))
    return df
df_submit = create_prediction(df_test, batch_size=64)
df_submit

In [0]:
plt.hist(df_submit['category'], bins=42, density=True)
plt.show()

In [0]:
df_submit.to_csv('submit.csv', index=False)